In [281]:
from clarifai.rest import ClarifaiApp
app = ClarifaiApp(api_key='97246cf0da164fd9963b504cccaca5ba')

In [290]:
from clarifai.rest import Image as CImage
import json



# this function returns the ingredients queried on clarifai
def get_ingreds(img_path,url_flag=1):
    model = app.models.get('food-items-v1.0')
    
    image = None
    if(not url_flag):
        image = CImage(file_obj=open(img_path,'rb'))
    else:
        image = CImage(url="https://samples.clarifai.com/metro-north.jpg")
        
    response = model.predict([image])
    parsed_response = json.loads(json.dumps(response))

    json_dict = parsed_response['outputs'][0]['data']['concepts']

    for i in json_dict:
        print(i)
        
    return json.dumps(json_dict)


# this function returns matching recipe names for the given response 

def get_matching_recipes(json_resp, recipes, url_flag = 1):
    
    # define matchingrecipes as finding the word in the list of recipes with some modifications*
    
    parsed_response = json.loads(json_resp)
    recipe_score = {}
    
    # apply tfidf --- read from offline file 
    tfidf = {}
    with open('all_stop_words.txt', 'r') as F:
        lines = F.readlines()
        for line in lines:
            words = line.replace('\n', '').split(' ')
            tfidf[words[0]] = int(words[1])
    
   # print(tfidf)
    
    for response in parsed_response:
        search_words = response['name'].lower().replace('-',' ').split(' ')
        #print('word in image :', search_words)
        for word in search_words:
            for ids,recipe_name in recipes.items():
                keywords = recipe_name.lower().replace('-',' ').split(' ')
                #print('recipe name is ', recipe_name)
                #print('keywords in recipe: ', keywords)
                if word in keywords:
                    #print('found word = ' , word, 'include ', recipe_name)
                    
                    word_score = 0
                    if(word in tfidf):
                        word_score = 1/tfidf[word]
                    else:
                        word_score = 1
                    
                    if(ids in recipe_score):
                        recipe_score[ids] += word_score
                    else:
                        recipe_score[ids] = word_score
                
                
        
    recipe_score = sorted(recipe_score.items(), key=lambda x: -x[1])
    return recipe_score
    
    
    #return list(set(matching_recipes))


In [291]:
def predict_ids(imgpath, url_flag=1):
    with open('recipes.json', 'r') as F:
        json_recipes = json.load(F)

    recipes = {}
    for data in json_recipes['recipes']:
        recipes[data['id']] = data['name']

    #print(recipes)
    matching_ids = get_matching_recipes(get_ingreds(imgpath,url_flag), recipes, url_flag)
    return matching_ids[:5]  # return top 5

In [292]:
predict_ids('images/turkey_cranberry.jpg',url_flag = 0)

{'id': 'ai_QLn2rxmZ', 'name': 'lettuce', 'value': 0.9741158485412598, 'app_id': 'main'}
{'id': 'ai_NhSzzDVx', 'name': 'sandwich', 'value': 0.9701259136199951, 'app_id': 'main'}
{'id': 'ai_FnZCSVMH', 'name': 'cheese', 'value': 0.9560813903808594, 'app_id': 'main'}
{'id': 'ai_w68d36Ks', 'name': 'bread', 'value': 0.9505338668823242, 'app_id': 'main'}
{'id': 'ai_CB8hsS3T', 'name': 'tomato', 'value': 0.9467867612838745, 'app_id': 'main'}
{'id': 'ai_jmcSl8c1', 'name': 'bacon', 'value': 0.9225103855133057, 'app_id': 'main'}
{'id': 'ai_KWmFf1fn', 'name': 'meat', 'value': 0.9212769865989685, 'app_id': 'main'}
{'id': 'ai_bq67BzDJ', 'name': 'toast', 'value': 0.9096548557281494, 'app_id': 'main'}
{'id': 'ai_H2d8CVB7', 'name': 'salad', 'value': 0.9089395999908447, 'app_id': 'main'}
{'id': 'ai_jvVxlhLh', 'name': 'chicken', 'value': 0.7502883076667786, 'app_id': 'main'}
{'id': 'ai_NDbbpCv1', 'name': 'vegetable', 'value': 0.748995304107666, 'app_id': 'main'}
{'id': 'ai_KF7Xcz9J', 'name': 'onion', 'val

[(2982, 0.9),
 (20679, 0.5833333333333334),
 (5388, 0.4),
 (6369, 0.4),
 (4858, 0.4)]